## Preparar el entorno y carga del modelo 

In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Mapea índice -> categoría (lo que usaste en training)
index_to_category = {
    0: "INDUSTRIAL", 
    1: "CIVIL", 
    2: "INFORMÁTICA",
    3: "TELECOMUNICACIONES",
    4: "ARQUITECTURA",
    5: "FILOSOFÍA",
    6: "PSICOLOGÍA",
    7: "LETRAS",
    8: "COMUNICACIÓN SOCIAL",
    9: "EDUCACIÓN",
    10: "ADMINISTRACIÓN",
    11: "CONTADURÍA",
    12: "RELACIONES INDUSTRIALES",
    13: "SOCIOLOGÍA",
    14: "ECONOMÍA",
    15: "DERECHO",
    16: "TEOLOGÍA"
}


# Dispositivo (GPU si está disponible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Cargar tokenizer
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Cargar modelo con la misma configuración
model = BertForSequenceClassification.from_pretrained(
    'dccuchile/bert-base-spanish-wwm-cased',
    num_labels=len(index_to_category)
)
model.load_state_dict(torch.load('best_model_state.bin', map_location=device))
model.to(device)
model.eval()  # Modo inferencia

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\aleja\AppData\Local\Temp\ipykernel_19200\207714581.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this m

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [2]:
## predict carrer

def predict_career(text_list, tokenizer, model, max_length=128):
    """
    text_list: lista de strings con las respuestas de usuario
    tokenizer: tokenizer de Hugging Face
    model: modelo BertForSequenceClassification cargado
    """
    # Tokenizar
    inputs = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # [batch_size, num_labels]
    
    # Obtener predicciones
    preds = torch.argmax(logits, dim=1).cpu().numpy()  # Índices de clase
    predicted_labels = [index_to_category[p] for p in preds]
    return predicted_labels



def predict_career_top3(text_list, tokenizer, model, index_to_category, 
                        device, max_length=128, top_k=3, threshold=0.5):
    """
    text_list: lista de strings con las respuestas de usuario
    tokenizer: tokenizer de Hugging Face
    model: modelo BertForSequenceClassification cargado
    index_to_category: diccionario {índice: categoría}
    device: 'cuda' o 'cpu'
    max_length: longitud máxima de tokenización
    top_k: número de predicciones (top) a retornar
    threshold: umbral mínimo de probabilidad para asignar una clase (vs 'UNKNOWN')
    """
    # Tokenizar
    inputs = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # [batch_size, num_labels]

        # Convertir logits a probabilidades con softmax
        probs = torch.softmax(logits, dim=1)  # [batch_size, num_labels]
        
        # Obtener los top_k índices y probabilidades
        top_probs, top_indices = probs.topk(top_k, dim=1, largest=True, sorted=True)
        # top_probs, top_indices => ([batch_size, top_k]), ([batch_size, top_k])

    batch_results = []
    for i in range(len(text_list)):
        # Extraer las k probabilidades e índices para esta fila i
        row_probs = top_probs[i].cpu().numpy()
        row_indices = top_indices[i].cpu().numpy()

        result = []
        for idx, p in zip(row_indices, row_probs):
            category_name = index_to_category[idx]
            result.append((category_name, float(p)))
        batch_results.append(result)

    return batch_results

In [ ]:
# Texto filtrado y concatenado
# Texto filtrado y concatenado
user_text = """Como desarrollador de IA, creo modelos para luego utilizarlos en aplicaciones. 
Me gustaba mucho la parte matemática, por ser algo práctico una vez entendías el tema. 
Sin embargo, también me gustaba la materia de literatura, historia y biología. 
Educación física, soy mala en los deportes. 
Si, un curso de inglés, me gusta mucho el idioma. 
Me gusta ver series y películas, también ver documentales o videos para aprender cosas nuevas de ciencia, historia, cultura general. 
Ciencia, tecnología, arte y cultura. 
Acerca de ciencia, como la teoría de la relatividad, cuántica, me gusta mucho entender el mundo desde esa perspectiva. 
Crear, investigar y resolver problemas. 
Me interesa entender cómo funciona el mundo, Me gusta imaginar y crear cosas nuevas, Prefiero resolver problemas prácticos y concretos. 
En un café. 
Tecnológico. 
Emprender."""




top3_preds = predict_career_top3(
    text_list=[user_text],
    tokenizer=tokenizer,
    model=model,
    index_to_category=index_to_category,
    device=device,
    top_k=3,
    threshold=0.5
)
print("Top 3 predicciones:", top3_preds[0])

Top 3 predicciones: [('FILOSOFÍA', 0.37672391533851624), ('INFORMÁTICA', 0.31185489892959595), ('SOCIOLOGÍA', 0.07941188663244247)]


In [4]:

"""import pandas as pd
from transformers import pipeline

# Cargar el dataset
input_path = "../data/raw/student_data_extended (2).csv"  # Reemplaza con la ruta a tu archivo
output_path = "../data/raw/student_data_extended_process.csv"  # Ruta para guardar el archivo final

df = pd.read_csv(input_path)

count = 0

# Modelos de traducción
model_name_es_en = "Helsinki-NLP/opus-mt-es-en"
model_name_en_es = "Helsinki-NLP/opus-mt-en-es"

translator_es_en = pipeline("translation_es_to_en", model=model_name_es_en, tokenizer=model_name_es_en)
translator_en_es = pipeline("translation_en_to_es", model=model_name_en_es, tokenizer=model_name_en_es)

# Función de traducción ida y vuelta
def back_translate(text):
    # Traducir de español a inglés
    translated_to_en = translator_es_en(text, max_length=512, truncation=True)
    text_in_english = translated_to_en[0]['translation_text']

    # Traducir de inglés a español
    translated_to_es = translator_en_es(text_in_english, max_length=512, truncation=True)
    text_in_spanish = translated_to_es[0]['translation_text']

    return text_in_spanish

# Crear las columnas `TEXTO` y `CARRERA`
processed_texts = []
processed_careers = []

for _, row in df.iterrows():
    # Combinar información relevante en una sola oración
    texto = f"estoy interesado en {row['AcademicInterest']}. " \
            f"Disfruta de {row['ExtracurricularActivities']}. " \
            f"Participo en actividades como {row['ClubMemberships']}. " \
            f"Pienso que tengo habilidades en {row['Skills']}. " \
            f"Tambien encuentro interes en {row['ResearchInterests']}."

    # Traducir el texto al español usando back-translation
    texto_traducido = back_translate(texto)

    processed_texts.append(texto_traducido)
    processed_careers.append(back_translate(row['AcademicInterest']))
    
    if count == 100:
        break
    count += 1
    print(count)

# Crear un nuevo DataFrame
df_processed = pd.DataFrame({
    "TEXTO": processed_texts,
    "CARRERA": processed_careers
})

# Guardar el dataset procesado
df_processed.to_csv(output_path, index=False, encoding="utf-8")

print(f"Dataset procesado guardado en: {output_path}")

"""

'import pandas as pd\nfrom transformers import pipeline\n\n# Cargar el dataset\ninput_path = "../data/raw/student_data_extended (2).csv"  # Reemplaza con la ruta a tu archivo\noutput_path = "../data/raw/student_data_extended_process.csv"  # Ruta para guardar el archivo final\n\ndf = pd.read_csv(input_path)\n\ncount = 0\n\n# Modelos de traducción\nmodel_name_es_en = "Helsinki-NLP/opus-mt-es-en"\nmodel_name_en_es = "Helsinki-NLP/opus-mt-en-es"\n\ntranslator_es_en = pipeline("translation_es_to_en", model=model_name_es_en, tokenizer=model_name_es_en)\ntranslator_en_es = pipeline("translation_en_to_es", model=model_name_en_es, tokenizer=model_name_en_es)\n\n# Función de traducción ida y vuelta\ndef back_translate(text):\n    # Traducir de español a inglés\n    translated_to_en = translator_es_en(text, max_length=512, truncation=True)\n    text_in_english = translated_to_en[0][\'translation_text\']\n\n    # Traducir de inglés a español\n    translated_to_es = translator_en_es(text_in_english

In [5]:
# Respuesta del estudiante 1
user_text1 = """Estoy interesado en Ingeniería Industrial. Fui becado y congelé mi carrera en 2020 por temas económicos. Me gustaban las materias de Matemática e Historia de Venezuela. No me agradaban Biología y Física. Me interesa resolver problemas y explorar culturas. Me gustaría ejercer mi carrera en España y viajar por el mundo."""

# Respuesta del estudiante 2
user_text2 = """Estoy interesado en Ingeniería Informática. Me gustaban Matemáticas y Deportes, eran las que mejores se me daban. No me agradaba Historia porque tengo mala memoria. Me interesa crear, ayudar y resolver problemas. Mi objetivo es conseguir empleo remoto en el extranjero."""

# Respuesta del estudiante 3
user_text3 = """Estoy interesada en Psicología. Me gustaban Física y Biología porque las entendía. No me agradaban Matemáticas y Castellano porque me parecían tediosas. Me gusta leer, ver películas y bailar. Me interesa crear e investigar. Mi meta es viajar y entender mejor el mundo."""

# Respuesta del estudiante 4
user_text4 = """Estoy interesada en Ingeniería Informática. Me gustaban Matemáticas, Química, Física y Contabilidad porque me gustan las cosas exactas. No me agradaban Dibujo, Historia y Geografía porque había que hacer muchas maquetas. Me gusta liderar, enseñar y resolver problemas. Mi objetivo es trabajar en una oficina."""

# Respuesta del estudiante 5
user_text5 = """Estoy interesado en Ingeniería Informática. Me gustaban Historia, Física y Química porque se me hacían divertidas. No me agradaba Castellano porque me parece aburrida. Me gusta jugar videojuegos y leer. Me interesa resolver problemas. Mi meta es ganar dinero para ayudar a mis padres."""

# Respuesta del estudiante 6
user_text6 = """Estoy interesado en Ingeniería en Informática. Me gustaban Matemáticas por sus propiedades y formas interesantes y Química por las reacciones y experimentos. No me agradaba Geografía porque no soy bueno ubicándome. Me gusta crear, liderar, ayudar y resolver problemas. Mi meta es desarrollar videojuegos."""

# Respuesta del estudiante 7
user_text7 = """Estoy interesada en Ingeniería Informática. Me gustaban Matemática y Física porque los números siempre me han llamado la atención. No me agradaba Química porque no entendía nada. Me interesa crear y explorar temas psicológicos y emocionales. Mi objetivo es manejarme bien y cocinar."""

# Respuesta del estudiante 8
user_text8 = """Estoy interesado en Ingeniería Informática. Me gustaban Matemáticas, Historia y Artes porque me interesan los cálculos, el pasado y el arte. No me agradaban Química y Geografía porque no las entendía bien. Me gusta dibujar, leer manga y jugar videojuegos. Mi meta es crear cómics."""

# Respuesta del estudiante 9
user_text9 = """Estoy interesada en Relaciones Industriales. Me gustaban Biología y Química porque me agradan sus funciones. No me agradaba Educación Física porque siempre terminaba con dolor de cabeza. Me gusta bailar y explorar culturas. Mi meta es que mi emprendimiento de dulces llegue a más personas."""

# Respuesta del estudiante 10
user_text10 = """Estoy interesada en Comunicación Social. Me gustaba Biología porque me ayudó a entender procesos humanos. No me agradaba Química porque no tuve un docente permanente. Me interesa ayudar y crear. Mi meta es emprender proyectos propios."""

# Respuesta del estudiante 11
user_text11 = """Estoy interesado en Comunicación Social. Me gustaban las materias relacionadas con Historia porque investigaba por mi cuenta. No me agradaban las Ciencias porque me costaban bastante. Me gusta leer, editar, y hacer fotografía. Mi meta es crear cosas útiles y vivir de ello."""

# Respuesta del estudiante 12
user_text12 = """Estoy interesada en Contaduría Pública. Me gustaban Contabilidad y Diseño porque me desenvolvía mejor. No me agradaban Matemática, Historia, Geografía y Política porque no me interesaban. Me gusta hacer ejercicio, pintar y bailar. Mi meta es ser bailarina profesional."""

# Respuesta del estudiante 13
user_text13 = """Estoy interesada en Educación. Me gustaban Matemática, Química, Biología y Arte porque las entendía bien. No me agradaba Física porque me enredaba en pequeñeces. Me gusta bailar, leer y hornear. Mi meta es abrir una pastelería y presentarme como bailarina profesional."""

# Respuesta del estudiante 14
user_text14 = """Estoy interesada en Ingeniería Informática. Me gustaban Matemáticas y Física porque eran interesantes. No me agradaba Química porque no me llamaba la atención. Me interesa resolver problemas y enseñar. Mi meta es levantar una empresa de desarrollo."""

# Respuesta del estudiante 15
user_text15 = """Estoy interesado en Ingeniería Informática. Me gustaba Computación porque me encantan las computadoras. No me agradaba Castellano porque no me gustan los idiomas. Me gusta leer y jugar videojuegos. Mi meta es tener mi propia oficina con un jardín."""

# Respuesta del estudiante 16
user_text16 = """Estoy interesado en Ingeniería en Sistemas. Me gustaba Química porque el profesor hacía los temas dinámicos. No me agradaba Historia de Venezuela porque era monótona. Me interesa enseñar y ayudar. Mi meta es aprender alemán y ser políglota."""

# Respuesta del estudiante 17
user_text17 = """Estoy interesado en Ingeniería Informática. Me gustaban Biología y Psicología porque me gusta entender el comportamiento. No me agradaba Química porque me costaba. Me gusta estar con mis amigos. Mi meta es conseguir estabilidad económica y aprender a tocar la guitarra."""

# Respuesta del estudiante 18
user_text18 = """Estoy interesada en Ingeniería Informática. Me gustaba Matemática porque es práctica y me gustan temas de ciencia. No me agradaba Educación Física porque soy mala en deportes. Me gusta ver series y aprender cosas nuevas. Mi meta es emprender."""

# Respuesta del estudiante 19
user_text19 = """Estoy interesada en Comunicación Social. Me gustaban Inglés, Biología y Psicología porque me sentía conectada con ellas. No me agradaban Física, Química y Matemática porque me costaba entenderlas. Me gusta investigar y enseñar. Mi meta es fundar una ONG para ayudar a mujeres y niños."""

# Respuesta del estudiante 20
user_text20 = """Estoy interesado en Física. Me gustaban Matemáticas y Física porque me gustan los cálculos. No me agradaba Dibujo Técnico porque no tenía paciencia. Me interesa crear y resolver problemas. Mi meta es graduarme de la universidad."""
